<a href="https://colab.research.google.com/github/utkarsh599/Recomendation-sysy/blob/Abhijeet/Major.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import spark
import nltk
import spacy
import pandas as pd
import sklearn 

In [2]:
from google.colab import files
Uploaded=files.upload()

Saving GB_category_id.json to GB_category_id.json
Saving GBcomments.csv to GBcomments.csv
Saving GBvideos.csv to GBvideos.csv
Saving US_category_id.json to US_category_id.json
Saving UScomments.csv to UScomments.csv
Saving USvideos.csv to USvideos.csv


In [43]:
import io
GB_category_id=pd.read_json(io.BytesIO(Uploaded["GB_category_id.json"]))
GBcomments=pd.read_csv(io.BytesIO(Uploaded["GBcomments.csv"]),error_bad_lines=False)
GBvideos=pd.read_csv(io.BytesIO(Uploaded["GBvideos.csv"]),error_bad_lines=False)
US_category_id=pd.read_json(io.BytesIO(Uploaded["US_category_id.json"]))
UScomments=pd.read_csv(io.BytesIO(Uploaded["UScomments.csv"]),error_bad_lines=False)
USvideos=pd.read_csv(io.BytesIO(Uploaded["USvideos.csv"]),error_bad_lines=False)



b'Skipping line 113225: expected 4 fields, saw 5\n'
b'Skipping line 158379: expected 4 fields, saw 7\nSkipping line 241590: expected 4 fields, saw 5\nSkipping line 245637: expected 4 fields, saw 7\n'
b'Skipping line 521402: expected 4 fields, saw 5\n'
b'Skipping line 2398: expected 11 fields, saw 21\nSkipping line 2797: expected 11 fields, saw 21\n'
b'Skipping line 41589: expected 4 fields, saw 11\nSkipping line 51628: expected 4 fields, saw 7\nSkipping line 114465: expected 4 fields, saw 5\n'
b'Skipping line 142496: expected 4 fields, saw 8\nSkipping line 189732: expected 4 fields, saw 6\nSkipping line 245218: expected 4 fields, saw 7\n'
b'Skipping line 388430: expected 4 fields, saw 5\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
b'Skipping line 2401: expected 11 fields, saw 21\nSkippi

In [44]:
GBcomments.head()

,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0


In [45]:
print(GBcomments.shape)
print(UScomments.shape)
print(GBvideos.shape)
print(USvideos.shape)

(718452, 4)
(691400, 4)
(7993, 11)
(7992, 11)


In [0]:
Comments=pd.concat([GBcomments,UScomments],ignore_index=True)
Videos=pd.concat([GBvideos,USvideos],ignore_index=True)

In [0]:
Comments.dropna(inplace=True)
Videos.dropna(inplace=True)

In [48]:
print(Comments.shape)
print(Videos.shape)

(1409799, 4)
(15985, 11)


In [0]:
Comments.comment_text=Comments.comment_text.astype(str)

In [50]:
Comments.head(10)


,video_id,comment_text,likes,replies
0,jt2OHQh0HoQ,It's more accurate to call it the M+ (1000) be...,0,0
1,jt2OHQh0HoQ,To be there with a samsung phone\n😂😂😂,1,0
2,jt2OHQh0HoQ,"Thank gosh, a place I can watch it without hav...",0,0
3,jt2OHQh0HoQ,What happened to the home button on the iPhone...,0,0
4,jt2OHQh0HoQ,Power is the disease. Care is the cure. Keep...,0,0
5,jt2OHQh0HoQ,Keep calm and buy iphone 8 Keep calm and buy i...,0,0
6,jt2OHQh0HoQ,i am a big fan of youtube and u !!!!!!!!!!!!!,0,0
7,jt2OHQh0HoQ,You will never find Losers who line up and pay...,0,0
8,jt2OHQh0HoQ,*APPLE JUST COMMENTED ON MY LAST VIDEO* I'm cr...,0,0
9,jt2OHQh0HoQ,"I'm only here to see Emma, I love her so much!...",0,0


In [51]:
Videos.head(10)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09
5,zZ2CLmvqfXg,"REVEALED - FIFA 18 stats for Chelsea's Hazard,...",Rio Ferdinand Presents FIVE,24,Rio Ferdinand|Rio|Ferdinand|5 Magazine|5 Mag|#...,317696,9449,135,464,https://i.ytimg.com/vi/zZ2CLmvqfXg/default.jpg,13.09
6,w8fAellnPns,Juicy Chicken Breast - You Suck at Cooking (ep...,You Suck At Cooking,26,how to|cooking|recipe|kitchen|chicken|chicken ...,479291,23935,638,1941,https://i.ytimg.com/vi/w8fAellnPns/default.jpg,13.09
7,LcZ2AuvxXNA,Nerf Bow Trick Shots | Dude Perfect,Dude Perfect,17,dude perfect|dude perfect stereotypes|dude per...,10532409,384841,7547,23849,https://i.ytimg.com/vi/LcZ2AuvxXNA/default.jpg,13.09
8,lamseWekh8Y,Weekend in BRIGHTON with Mike! | Fleur De Vlog,FleurDeVlog,22,fleurdevlog|fleur de vlog|fleurdeforce|fleur d...,65453,2761,33,223,https://i.ytimg.com/vi/lamseWekh8Y/default.jpg,13.09
9,QBGaO89cBMI,Radiohead - Lift,Radiohead,10,radiohead|lift|ok computer|oknotok,751743,42272,358,3250,https://i.ytimg.com/vi/QBGaO89cBMI/default.jpg,13.09


In [0]:
Comments['comment_text']=Comments['comment_text'].apply(lambda x: x.lower())

In [0]:
sampled_comments=Comments.sample(30000)

In [54]:
sampled_comments.head()

,video_id,comment_text,likes,replies
682624,EUoe7cf0HYw,i thought she was dead,0,0
587297,fBR-MFpVKuc,daddy evans,0,0
850388,2m65xSqUWaI,and to think i let my dog give birth on that t...,0,0
549948,iEqgI3iwI2U,grahams reaction hahah,0,0
1422,6vGg-jJl30A,congratulations!! nice to meet your channel\n\...,0,0


# Tokenization using Spacy

## [Click here for Documentation of spacymoji](https://github.com/ines/spacymoji)

In [0]:
# pip install spacymoji

In [0]:
# from spacymoji import Emoji
# emoji = Emoji(nlp)

In [57]:
nlp = spacy.load("en_core_web_sm")
#nlp.add_pipe(emoji, first=True)
c=0
tokenized_sents=[]
for i in sampled_comments['comment_text']:
    doc=nlp(i)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    tokenized_sents.append(tokens)
    print(c,end='\r')
    c+=1

In [0]:
#sampled_comments.drop('tokenized_comments',inplace=True,axis=1)

In [0]:
sampled_comments['tokenized_comments']=tokenized_sents

In [60]:
sampled_comments.head()

,video_id,comment_text,likes,replies,tokenized_comments
682624,EUoe7cf0HYw,i thought she was dead,0,0,"[think, dead]"
587297,fBR-MFpVKuc,daddy evans,0,0,"[daddy, evan]"
850388,2m65xSqUWaI,and to think i let my dog give birth on that t...,0,0,"[think, let, dog, birth, thing]"
549948,iEqgI3iwI2U,grahams reaction hahah,0,0,"[graham, reaction, hahah]"
1422,6vGg-jJl30A,congratulations!! nice to meet your channel\n\...,0,0,"[congratulation, nice, meet, time, han, zimmer..."


In [0]:
tokenized_sents=[]
for i in sampled_comments['tokenized_comments']:
  tokenized_sents.append(" ".join(i))

In [62]:
sampled_comments.drop('tokenized_comments',inplace=True,axis=1)
sampled_comments['tokenized_comments']=tokenized_sents
sampled_comments.head()

,video_id,comment_text,likes,replies,tokenized_comments
682624,EUoe7cf0HYw,i thought she was dead,0,0,think dead
587297,fBR-MFpVKuc,daddy evans,0,0,daddy evan
850388,2m65xSqUWaI,and to think i let my dog give birth on that t...,0,0,think let dog birth thing
549948,iEqgI3iwI2U,grahams reaction hahah,0,0,graham reaction hahah
1422,6vGg-jJl30A,congratulations!! nice to meet your channel\n\...,0,0,congratulation nice meet time han zimmer song ...


In [64]:
# sampled_df.drop(['likes','replies'],inplace=True,axis=1)
 sampled_comments.head()

,video_id,comment_text,likes,replies,tokenized_comments
682624,EUoe7cf0HYw,i thought she was dead,0,0,think dead
587297,fBR-MFpVKuc,daddy evans,0,0,daddy evan
850388,2m65xSqUWaI,and to think i let my dog give birth on that t...,0,0,think let dog birth thing
549948,iEqgI3iwI2U,grahams reaction hahah,0,0,graham reaction hahah
1422,6vGg-jJl30A,congratulations!! nice to meet your channel\n\...,0,0,congratulation nice meet time han zimmer song ...


In [65]:
pip install nltk

In [0]:
import nltk

In [68]:
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [0]:
words = set(nltk.corpus.words.words())

cleaned_sents=[]

for sent in sampled_comments['tokenized_comments']:
  cleaned_sents.append(" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha()))

In [71]:
sampled_comments.drop('tokenized_comments',inplace=True,axis=1)
sampled_comments['tokenized_comments']=cleaned_sents
sampled_comments.head()

,video_id,comment_text,likes,replies,tokenized_comments
682624,EUoe7cf0HYw,i thought she was dead,0,0,think dead
587297,fBR-MFpVKuc,daddy evans,0,0,daddy
850388,2m65xSqUWaI,and to think i let my dog give birth on that t...,0,0,think let dog birth thing
549948,iEqgI3iwI2U,grahams reaction hahah,0,0,graham reaction
1422,6vGg-jJl30A,congratulations!! nice to meet your channel\n\...,0,0,congratulation nice meet time han song amazing...


In [29]:
Videos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [0]:
Videos['tags']=Videos['tags'].apply(lambda x: x.split("|"))

In [0]:
Videos['tags']=Videos['tags'].apply(lambda x: " ".join(x))

In [41]:
Videos.head()

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,jt2OHQh0HoQ,Live Apple Event - Apple September Event 2017 ...,Apple Event,28,apple events apple event iphone 8 iphone x iph...,7426393,78240,13548,705,https://i.ytimg.com/vi/jt2OHQh0HoQ/default_liv...,13.09
1,AqokkXoa7uE,Holly and Phillip Meet Samantha the Sex Robot ...,This Morning,24,this morning interview holly willoughby philli...,494203,2651,1309,0,https://i.ytimg.com/vi/AqokkXoa7uE/default.jpg,13.09
2,YPVcg45W0z4,My DNA Test Results! I'm WHAT?!,emmablackery,24,emmablackery emma blackery emma blackery briti...,142819,13119,151,1141,https://i.ytimg.com/vi/YPVcg45W0z4/default.jpg,13.09
3,T_PuZBdT2iM,getting into a conversation in a language you ...,ProZD,1,skit korean language conversation esl japanese...,1580028,65729,1529,3598,https://i.ytimg.com/vi/T_PuZBdT2iM/default.jpg,13.09
4,NsjsmgmbCfc,Baby Name Challenge!,Sprinkleofglitter,26,sprinkleofglitter sprinkle of glitter baby gli...,40592,5019,57,490,https://i.ytimg.com/vi/NsjsmgmbCfc/default.jpg,13.09


In [0]:
Videos['tags']=Videos['tags'].apply(lambda x: x.lower())

In [84]:
nlp = spacy.load("en")
c=0
tokenized_sents=[]
for i in Videos['tags']:
    doc=nlp(i)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    tokenized_sents.append(tokens)
    print(c,end='\r')
    c+=1

In [0]:
Videos.drop(['tokenized_tags'],axis=1,inplace=True)
Videos['tokenized_tags']=tokenized_sents

In [87]:
Videos.head()

,video_id,title,category_id,tags,views,likes,dislikes,comment_total,tokenized_titles,tokenized_tags
0,jt2OHQh0HoQ,live apple event - apple september event 2017 ...,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,"[live, Apple, Event, Apple, September, event, ...",[apple]
1,AqokkXoa7uE,holly and phillip meet samantha the sex robot ...,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,"[Holly, Phillip, Meet, Samantha, Sex, Robot, m...","[show, show, life]"
2,YPVcg45W0z4,my dna test results! i'm what?!,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,"[dna, Test, Results]",[]
3,T_PuZBdT2iM,getting into a conversation in a language you ...,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,"[get, conversation, language, actually, speak]",[]
4,NsjsmgmbCfc,baby name challenge!,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,"[Baby, Challenge]","[pentland, baby, baby, baby, name]"


In [0]:
Videos['tokenized_tags']=Videos['tokenized_tags'].apply(lambda x: list(dict.fromkeys(x)))

# mylist = ["a", "b", "a", "c", "c"]
# mylist = list(dict.fromkeys(mylist))
# print(mylist)


In [0]:
Videos['title']=Videos['title'].apply(lambda x: x.lower())

In [0]:
#Videos.drop(['thumbnail_link','date','channel_title'],axis=1,inplace=True)

In [90]:
nlp = spacy.load("en")
c=0
tokenized_sents=[]
for i in Videos['title']:
    doc=nlp(i)
    tokens=[]
    for token in doc:
      if((token.is_stop==False and token.is_punct==False) and token.is_alpha==True):
        tokens.append(token.lemma_)
    tokenized_sents.append(tokens)
    print(c,end='\r')
    c+=1

In [0]:
Videos['tokenized_titles']=tokenized_sents

In [92]:
Videos.head()

,video_id,title,category_id,tags,views,likes,dislikes,comment_total,tokenized_titles,tokenized_tags
0,jt2OHQh0HoQ,live apple event - apple september event 2017 ...,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,"[live, apple, event, apple, september, event, ...",[apple]
1,AqokkXoa7uE,holly and phillip meet samantha the sex robot ...,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,"[holly, phillip, meet, samantha, sex, robot, m...","[show, life]"
2,YPVcg45W0z4,my dna test results! i'm what?!,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,"[dna, test, result]",[]
3,T_PuZBdT2iM,getting into a conversation in a language you ...,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,"[get, conversation, language, actually, speak]",[]
4,NsjsmgmbCfc,baby name challenge!,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,"[baby, challenge]","[pentland, baby, name]"


In [0]:
tokenized_sents=[]
for i in Videos['tokenized_titles']:
  tokenized_sents.append(" ".join(i))

In [96]:
Videos.drop('tokenized_titles',inplace=True,axis=1)
Videos['tokenized_titles']=tokenized_sents
Videos.head()

,video_id,title,category_id,tags,views,likes,dislikes,comment_total,tokenized_tags,tokenized_titles
0,jt2OHQh0HoQ,live apple event - apple september event 2017 ...,28,apple events|apple event|iphone 8|iphone x|iph...,7426393,78240,13548,705,[apple],live apple event apple september event iphone ...
1,AqokkXoa7uE,holly and phillip meet samantha the sex robot ...,24,this morning|interview|holly willoughby|philli...,494203,2651,1309,0,"[show, life]",holly phillip meet samantha sex robot morning
2,YPVcg45W0z4,my dna test results! i'm what?!,24,emmablackery|emma blackery|emma|blackery|briti...,142819,13119,151,1141,[],dna test result
3,T_PuZBdT2iM,getting into a conversation in a language you ...,1,skit|korean|language|conversation|esl|japanese...,1580028,65729,1529,3598,[],get conversation language actually speak
4,NsjsmgmbCfc,baby name challenge!,26,sprinkleofglitter|sprinkle of glitter|baby gli...,40592,5019,57,490,"[pentland, baby, name]",baby challenge


In [0]:
All_tags=[]
for i in Videos['tokenized_tags']:
  All_tags.extend(i)

In [100]:
All_tags

['apple',
 'show',
 'life',
 'pentland',
 'baby',
 'name',
 'rio',
 'ferdinand',
 'fifa',
 'luiz',
 'pack',
 'official',
 'rating',
 'chicken',
 'dude',
 'perfect',
 'water',
 'bottle',
 'world',
 'trick',
 'shot',
 'de',
 'force',
 'jame',
 'late',
 'night',
 'harry',
 'lounge',
 'competition',
 'market',
 'actually',
 'plane',
 'neo',
 'nile',
 'wilson',
 'gymnastic',
 'dad',
 'start',
 'world',
 'smart',
 'real',
 'makeup',
 'healthy',
 'care',
 'heart',
 'stitch',
 'tower',
 'lilo',
 'change',
 'culture',
 'response',
 'pop',
 'rooster',
 'life',
 'interesting',
 'child',
 'year',
 'old',
 'selling',
 'novel',
 'worth',
 'patrol',
 'get',
 'family',
 'school',
 'de',
 'nyma',
 'dark',
 'contour',
 'beauty',
 'pro',
 'filter',
 'foundation',
 'tang',
 'fenty',
 'makeup',
 'wife',
 'bank',
 'august',
 'favourite',
 'murphy',
 'mini',
 'tracking',
 'grey',
 'makeup',
 'x',
 'revolution',
 'eyeshadow',
 'sophdoesnail',
 'soph',
 'palette',
 'drugstore',
 'wrestle',
 'ty',
 'st',
 'scp'